In [197]:

import sys
import csv
import math
import copy
import time
import numpy as np
from collections import Counter
from numpy import *
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'ignore', 'invalid': 'ignore', 'over': 'warn', 'under': 'ignore'}

## Multiclass classifier decision tree using ID3 algorithm

In [198]:
feature = []

In [199]:
#normalize the entire dataset prior to learning using min-max normalization 
def normalize(matrix):
#     transfer the data metrix to np array in float type.
    a = np.array(matrix)
    a = a.astype(float)
    
    # print("normalizing the entire dataset:")
    # print(a)
    # print("Before normalizing")
#     apply the normalization along the 0 axis of a using the formula: (x - x_min)/(x_max - x_min)
    
   
    a_min = a.min(axis=0)
    a_max = a.max(axis=0)
    val = (a - a_min)/(a_max - a_min)
    return val


In [200]:
# reading from the file using numpy genfromtxt with delimiter ','
def load_csv(file):
  X = np.genfromtxt(file, delimiter=",", dtype=str)
  return (X)

In [201]:
#method to randomly shuffle the array using the numpy.random.shuffle()
def random_numpy_array(ar):
  np.random.shuffle(ar)
  arr = ar
  return arr

In [202]:
#Normalize the data and generate the training labels,training features, test labels and test training
def generate_set(X):
    print(X.shape[0])
#     store the label X[:,-1] to Y
    Y = X[:,-1]

#     reshape y to (Y's length, 1)
#     store it to j
    j = Y.reshape((len(Y),1))

    # print("J is",j)
#     create the new_X which exclude the label X[:,:-1]
    new_X = X[:,:-1]

#     normalize the data step
#     using our implemented function normalize()
    norm_X = normalize(new_X)

#     add the label back to the normiazlied X
#     using np.concatenate along axis=1
    X_after_norm = np.concatenate((norm_X, j), axis=1)


#     store the size of rows of the normalized X with labels
    rows_size = X_after_norm.shape[0]


#     use the 10% of the data to be the test set.
#     store the number of testing data
    num_test_data = round(0.1 * rows_size)

#     set the starting idex to be 0
    start = 0

#     set the ending index to be the number of testing data
    end = num_test_data

#     create a list that store all features of the testing data
    features_testData_list = []

#     create a list that store all labels of the testing data
    labels_testData_list = []

#     create a list that store all features of the training data
    features_trainData_list = []
    
#     create a list that store all labels of the training data
    labels_trainData_list = []
#     10-fold cross-validation:
    for i in range(10):
#         store the test set for corss-validation using X[start:end,:]
      X_after_norm_test = X_after_norm[start:end,:]

#         get training data before the testing data X[:start, :]
      train_bef_test = X_after_norm[:start,:]

#         get training data after the testing data X[end:, :]
      train_aft_test = X_after_norm[end:,:]

#         form the new training set using np.concatenate
      X_after_norm_training = np.concatenate((train_bef_test,train_aft_test),axis=0)

#         get the testing set labels
      test_set_labels = X_after_norm_test[:,-1]

#         flattent the labels
      test_set_labels = test_set_labels.flatten()

#         get the training set labels
      train_set_labels = X_after_norm_training[:,-1]

#         flattent the labels
      train_set_labels = train_set_labels.flatten()

#         create the test set exclude the labels
      X_after_norm_test = X_after_norm_test[:,:-1]
      X_after_norm_test = X_after_norm_test.astype(float)


#         same for the training set
      X_after_norm_training = X_after_norm_training[:,:-1]
      X_after_norm_training = X_after_norm_training.astype(float)

#         append test data of this fold to the list
      features_testData_list.append(X_after_norm_test)


#         append test lables of this fold to the list
      labels_testData_list.append(test_set_labels)

#         do the same for the training set
      features_trainData_list.append(X_after_norm_training)
      labels_trainData_list.append(train_set_labels)


#         update the index pointer
      start = end 
      end = end + num_test_data


#     return the fold list that contain data and label for both training and testing set.
    return features_testData_list, labels_testData_list, features_trainData_list, labels_trainData_list


In [203]:
#build a dictionary where the key is the class label and values are the features which belong to that class.
def build_dict_of_attributes_with_class_values(X,y):
#     init a dict for attributes
  attributes_dictionary = {}

#     init feature list.
  fetaure_list = []

#     for each feature in the dataset
  for i in range(X.shape[1]):
#         store the featur index
    feature_idx = i

#     find all the value correspond to this feature
    val = X[:,i]

#     init an attribute list
    attribute_list = []

#     init the counter to 0
    cnt = 0

#         for each value in the "all the value correspond to this feature"
    for element in val:
#             init a empty list that store the attribute value
      attribute_val = []

#             append the this value to the list
      attribute_val.append(element)

#             append the label of this value to the list
      attribute_val.append(y[cnt])

#             append this list to the attribute list.
      attribute_list.append(attribute_val)

#             increase the counter
      cnt = cnt + 1

#         add this attribute list to the dict according to the feature index
    attributes_dictionary[feature_idx] = attribute_list

#         append the feature indx to the feature list.
    fetaure_list.append(feature_idx)

#     return the dict and feature list.
  return attributes_dictionary, fetaure_list


In [204]:
# Y = {'a':1, 'b':2}
# if ( 'a' in Y):
#   print(Y['a'])

In [205]:
# Iterative Dichotomiser 3 entropy calculation
def entropy(y):
#     init a class frequence dict
  class_frequency_dictionary = {}

#     init the attribute entropy to 0
  attribute_entropy = 0

#     for each label in y:
  for i in y:
#         if this is label is already in the dict, we increase its feq
    if (i in class_frequency_dictionary):
      class_frequency_dictionary[i] +=1

#          else, we set the freq to 1
    else:
      class_frequency_dictionary[i] = 1


#     calculate the cumulate entropy using the formula.
    for val in class_frequency_dictionary.values():
      p = val/float(len(y))

      attribute_entropy += ((-p) * math.log(p,2))


  return attribute_entropy


In [206]:
#Class node and explanation is self explaination
class Node(object):
#     init the node with val,lchild,rchild,theta and leaf.
    def __init__(self, val, lchild, rchild,theta,leaf):
        self.root_value = val
        self.root_left = lchild
        self.root_right = rchild
        self.theta = theta
        self.leaf = leaf

#     method to identify if the node is leaf
    def is_leaf(self):
        
        return self.leaf

#     method to return threshold value
    def ret_thetha(self):
        
        return self.theta
    
#     method return root value
    def ret_root_value(self):
        
        return self.root_value
    
#     method return left tree
    def ret_llist(self):
        
        return self.root_left

#     method return right tree
    def ret_rlist(self):
        
        return self.root_right

    def __repr__(self):
        return "(%r, %r, %r, %r)" %(self.root_value,self.root_left,self.root_right,self.theta)


In [207]:
#Decision tree object
class DecisionTree(object):
#     init a variable called fea_list
  fea_list = []

#     init the Dtree by setting the root node to None
  def __init__(self):
      self.root_node = None

  #method to return the major class value using Counter() and .most_common()
  def cal_major_class_values(self,class_values):
    val = Counter(class_values)
    val = val.most_common(1)[0][0]
  
    return val 

  #method to calculate best threshold value for each feature
  def cal_best_theta_value(self,ke,attri_list):
#         init a list for data
    val = []

#         init a list for class labes
    class_values = []


#         for each attribute in the attri_list
    for i in attri_list:
#             append the data
      val.append(i[0])

#             append the feature value.
      class_values.append(i[1])

#         calculate the entropy of those feaure values
    entropy_feature_i = entropy(class_values)

#         init the max info gain = 0
    max_info_gain = 0

#         init theta=0
    theta = 0

#         init a list that store the best index on the left
    best_index_on_left_list = []

#         init a list that store the best index on the right
    best_index_on_right_list = []

#         init a list that store class labels after split
    class_labels_after_split_list = []


#         sort the data
    val.sort()

#         for each index of data:
    for idx in range(len(val)-1):


#             calculate the current theta using data[i]+data[i+1])/ 2
      current_theta = float(val[idx]+val[idx+1])/ 2

#             init a list that store index that less than theta
      index_less_than_theta_list = []

#             init a list that store value that less than theta
      values_less_than_theta_list = []

#             init a list that store index that greater than theta
      index_greater_than_theta_list = []

#             init a list that store value that less than theta
      values_greater_than_theta_list = []

#             init the counter to 0
      cnt = 0

#             for each index and value in attri_list
      for c,j in enumerate(attri_list):
#                 if value less or equal than the current theta:
        if (j[0] <= current_theta):


#                     update the "less" list of index and value
          index_less_than_theta_list.append(c)
          values_less_than_theta_list.append(j[1])

#                 else:
        else:
#                     update the "greater" list of index and value
          index_greater_than_theta_list.append(c)
          values_greater_than_theta_list.append(j[1])
#             calculate the entropy of the "less" list
      less_list_entropy = entropy(values_less_than_theta_list)
#             calculate the entropy of the "greater" list
      greater_list_entropy = entropy(values_greater_than_theta_list)
#             calculate the info gain using the formular.

      num1 = len(index_less_than_theta_list)
      num2 = len(index_greater_than_theta_list)
      num3 = len(attri_list)

      current_info_gain = entropy_feature_i - (less_list_entropy * (num1/float(num3))) - (greater_list_entropy * (num2/float(num3)))
#             if current info gain > max info gan
      if ( current_info_gain > max_info_gain):
        
#                 update the info gain, 
        max_info_gain = current_info_gain
#                     the theta, the best index list of right, 
        theta = current_theta
#                         the best index list of left and class_labels_list_after_split
        best_index_on_left_list = index_less_than_theta_list
        best_index_on_right_list = index_greater_than_theta_list

        class_labels_after_split_list = values_greater_than_theta_list + values_less_than_theta_list





#         return the max info gain, theata,the best left list,the best right list and class label after split
    return max_info_gain, theta, best_index_on_left_list, best_index_on_right_list, class_labels_after_split_list

#method to select the best feature out of all the features.
  def best_feature(self,dict_rep):
        
#         set key value to none
    key_value = None
#         set best info gain to -1
    best_info_gain = -1 

#         set best theta to 0
    best_theta = 0

#         set best left list to empty
    best_index_on_left_list = []

#         set best right list to empty
    best_index_on_right_list = []

#         set best class labels after split to empty
    best_class_labels_after_split_list = []

#         init a result list
    result_list = []


#         for each key in dict_rep:
    for ke in dict_rep.keys():
#             using cal_best_theta_value() and store all returned values
      info_gain, theta, index_on_left_list, index_on_right_list, class_labels_after_split_l = self.cal_best_theta_value(ke, dict_rep[ke])

#             if info gian is greater than best info gain:
      if (info_gain > best_info_gain):
#                 update info gain, theth, key value,
#                 left list, right list, class labels after split
        best_info_gain = info_gain
        best_theta = theta
        key_value = ke
        best_index_on_left_list = index_on_left_list
        best_index_on_right_list = index_on_right_list
        best_class_labels_after_split_list = class_labels_after_split_l

#         append the key value to the retrun list
    result_list.append(key_value)

#         append the theta value to the retrun list
    result_list.append(best_theta)

#         append the left list to the retrun list
    result_list.append(best_index_on_left_list)

#         append the right list to the retrun list
    result_list.append(best_index_on_right_list)

#         append the class labels to the retrun list
    result_list.append(best_class_labels_after_split_list)

#         return the list.
    return result_list

  def get_remainder_dict(self,dict_of_everything,index_split):
    global fea_list
#         init a split dict
    split_dict = {}

#         for each key "ke" in dict_of_everything:
    for ke in dict_of_everything.keys():
#             init a value list
      value_list = []

#             init a modified list
      modified_list = []

#             get the corresponding values of the key"ke" 
      corresp_val_of_keys = dict_of_everything[ke]

#             for each value and its corresponding index of the key"ke" 
      # for index, value in enumerate(corresp_val_of_keys):
      for each_val in range(len(corresp_val_of_keys)):
#                 if index is not in the index_split:
        if ( each_val not in index_split):

#                     append it to the modified list and value list
          modified_list.append(corresp_val_of_keys[each_val])
          value_list.append(corresp_val_of_keys[each_val][1]) # 1 ho ki i
#             add this modified list to the dict
      split_dict[ke] = modified_list
      key_name = split_dict.keys()

#         return the splited dict and val list
    return split_dict, value_list

  #method to create decision tree
  def create_decision_tree(self, dict_of_everything,class_val,eta_min_val):

    global fea_list
    #if all the class labels are same, then we are set
    if (len(set(class_val)) ==1):
      # print("Leaf node for set class is",class_val[0],len(class_val))

      # root_node = Node(majority_val, None, None, 0, True)
      p = Node(class_val[0], None, None, 0, True)

        
      return p
    #if the no class vales are less than threshold, we assign the class with max values as the class label    
    elif (len(class_val) < eta_min_val):
      majority_val = self.cal_major_class_values(class_val)
      q = Node(majority_val, None, None, 0, True)
      return q          

    else:
#             using the best_feature to get best feature list
      best_features_list = self.best_feature(dict_of_everything)
#             store the node name, theta,left split, right split and class labes
      node_name = best_features_list[0]
      theta = best_features_list[1]
      left_split = best_features_list[2]
      right_split = best_features_list[3]
      class_labels = best_features_list[4]
      # print("Hello")

#             call get_remainder_dict to get left tree data
      left_dict, class_value_1 = self.get_remainder_dict(dict_of_everything, left_split)

#             call get_remainder_dict to get right tree data
      right_dict, class_value_2 = self.get_remainder_dict(dict_of_everything, right_split)

#             call create_decision_tree to get left tree based on the left tree data
      
      leftChild = self.create_decision_tree(left_dict, class_value_1,eta_min_val)
      # leftChild = None

#             call create_decision_tree to get right tree based on therightleft tree data
      rightChild = self.create_decision_tree(right_dict, class_value_2,eta_min_val)
     
      # print("LeftChild ", leftChild)
      # print("rightChild ", rightChild)
#             set the root node
      root_node = Node(node_name, rightChild, leftChild, theta, False)
      # root_node = Node(node_name, theta, False)
      # print("root node ", root_node)


#             return root node
      return root_node
        
  #fit the decisin tree
  def fit(self, dict_of_everything,cl_val,features,eta_min_val):
#         set the fea_list the value of features
    global fea_list
    fea_list = features   
#         set the root node using the function create_decision_tree()
    root_node = self.create_decision_tree(dict_of_everything, cl_val, eta_min_val)

    return root_node

  def classify(self,row,root):
#         init the test dict
    test_dict = {}

#         add row to the dict
    for k,j in enumerate(row):
      test_dict[k] = j
#         set the current node to root
    current_node = root
      

#         while the current node is not leaf:
    while not current_node.leaf:
#             implement the case whether the current shoud go to the left
      if (test_dict[current_node.root_value]) <= current_node.theta:
        current_node = current_node.root_left

#             implement the case whether the current shoud go to the right
      else:
        current_node = current_node.root_right

    
#         return the calss of the current node
    return current_node.root_value
      
  #method to the labels for the test data
  def predict(self, X, root):
    predict_list = []
    for data in X:
#         predict using the classify()
      predicted_val = self.classify(data,root)
      predict_list.append(predicted_val)
    return predict_list

In [208]:
#calculating the predicited accuracy
def accuracy_for_predicted_values(test_class_names1,l):
#     init true and false count to 0
  true_cnt = 0
  false_cnt = 0

#     for each prediction,if predict is correct then, true++ else, false++
  for i in range(len(test_class_names1)):
    if (test_class_names1[i]== l[i]):
      true_cnt += 1
      
    else:
      false_cnt += 1   
      
#     return the acc
  return true_cnt, false_cnt, float(true_cnt)/len(l)

In [209]:
def main(num_arr, eta_min):

  eta_min_val = round(eta_min*num_arr.shape[0])
  #randomly shuffle the array so that we can divide the data into test/training
  random_array_1 = random_numpy_array(num_arr)
  
  #divide data into test labels,test features,training labels, training features
  features_testData_list, labels_testData_list, features_trainData_list, labels_trainData_list = generate_set(random_array_1)
  
#     init cumulate acc to 0
  cumulative_acc = 0
# standard deviation 
  standard_deviation_list = []
  #ten fold iteration 
  for i in range(10):
  
    #build a dictionary with class labels and respective features values belonging to that class
    attributes_dictionary, fetaure_list = build_dict_of_attributes_with_class_values(features_trainData_list[i], labels_trainData_list[i])
    
    #instantiate decision tree instance
    decision_tree_dict = DecisionTree()
    
    # build the decision tree model.
    decision_tree_model = decision_tree_dict.fit(attributes_dictionary,labels_trainData_list[i],fetaure_list,eta_min_val)
    
    #predict the class labels for test features
    class_labels_pred = decision_tree_dict.predict(features_testData_list[i],decision_tree_model)
    
    #calculate the accuracy for the predicted values 
    right, wrong, accu = accuracy_for_predicted_values(labels_testData_list[i],class_labels_pred)   
    
#         add acc to cumulate acc
    cumulative_acc += accu
    standard_deviation_list.append(accu)

    print("Accuracy is ",accu)
  print("Accuracy across 10-cross validation for ",eta_min," is ",float(cumulative_acc)/10)
  sum = 0
  for e in standard_deviation_list:
    sum = sum + (float(e) - float(cumulative_acc)/10)**2
  print("Standard deviation across 10-cross validation for ",eta_min," is ", (sum/(10-1))**0.5)
  print("\n")





In [210]:
eta_min_list = [0.05,0.10,0.15,0.20,0.25]
# newfile = "/spambase.csv"
file_name = input("Please press 1 for 'iris.csv' and 2 for 'spambase.csv': ")
print("\n")
if (file_name == '1'):
  newfile = "/iris.csv"
elif (file_name == '2'):
  newfile = "/spambase.csv"

else:
  print("Wrong input!")


num_arr = load_csv(newfile)
for i in eta_min_list:
  main(num_arr,i)

Please press 1 for 'iris.csv' and 2 for 'spambase.csv': 1


150
Accuracy is  0.9333333333333333
Accuracy is  0.9333333333333333
Accuracy is  1.0
Accuracy is  0.8666666666666667
Accuracy is  0.9333333333333333
Accuracy is  0.9333333333333333
Accuracy is  0.8
Accuracy is  1.0
Accuracy is  0.8666666666666667
Accuracy is  0.8666666666666667
Accuracy across 10-cross validation for  0.05  is  0.9133333333333334
Standard deviation across 10-cross validation for  0.05  is  0.06324555320336757


150
Accuracy is  0.9333333333333333
Accuracy is  0.9333333333333333
Accuracy is  0.8
Accuracy is  1.0
Accuracy is  1.0
Accuracy is  0.9333333333333333
Accuracy is  0.9333333333333333
Accuracy is  1.0
Accuracy is  0.7333333333333333
Accuracy is  1.0
Accuracy across 10-cross validation for  0.1  is  0.9266666666666667
Standard deviation across 10-cross validation for  0.1  is  0.09135468796041986


150
Accuracy is  1.0
Accuracy is  1.0
Accuracy is  0.8666666666666667
Accuracy is  0.9333333333333333
Accura